In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import sys

from tensorflow import keras

sys.path.append('..')
import model
import utils
import preprocessing

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])
params['batch_size'] = 10

In [18]:
def make_datagenerator(params, mode):
    
    file_path = mode + '.txt'
    data_path = os.path.join(params['data_dir'],file_path)
    
    df = pd.read_csv(data_path, sep="\t", header=0)
    
    datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_dataframe(
        dataframe = df,
        x_col = 'images',
        target_size = params['image_shape'],
        batch_size = params['batch_size'],
        class_mode = None)
    
    return generator

In [19]:
def load_model(params):
    #tf.keras.backend.clear_session()
    width, height = params['image_shape']
    inputs = tf.keras.layers.Input(shape=(width, height, 3))
    net = model.ModelArchitecture(num_classes=params['num_classes'])
    x = net(inputs, training=False)
    return net
    

def make_predictions(generator, params):
    
    net = load_model(params)
    
    optimizer = tf.keras.optimizers.Adam(lr=params['learning_rate'])
    net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
    net.load_weights(os.path.join(params['model_dir'], 'tf_ckpt'))
    #predictions = net.predict(x=data_to_predict, batch_size=params['batch_size'], verbose=1)
    predictions = net.predict_generator(generator)
    
    return np.argmax(predictions,axis=1)

In [20]:
def plot_results(data_to_predict,predictions, params):
    
    id_label_map = utils.load_id_label_map(params)
    
    num_audios = len(data_to_predict) if len(data_to_predict)>1 else len(data_to_predict)+1
    
    fig, axis = plt.subplots(num_audios,2, figsize=(20,20))
    
    images = data_to_predict[0]
    
    k = 0
    for i in range(len(data_to_predict)):

        axis[i][0].imshow(data_to_predict[k])
        axis[i][0].axis('off')

        pred = id_label_map[predictions[k]]
        axis[i][1].text(0.5, 0.5, pred, size=25, ha="center", va="center",
                bbox=dict(
                    boxstyle="round",
                    ec=(1., 0.5, 0.5),
                    fc=(1., 0.8, 0.8),
                )
               )
        axis[i][1].set_title('Prediction')
        axis[i][1].axis('off')
            
        k += 1
    plt.show()

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

def vudu_preds(params):
    
    """
    
    images_path = os.path.join(params['data_dir'], 'VUDU')
    label_id_map = preprocessing._load_label_id_map(params)
    
    images_list = list()
    label_list = list()

    for root, dirs, files in os.walk(images_path, topdown=False):
        for name in files:

            image_name = os.path.join(root, name)
            label_class = name[name.index('-')+1:name.rindex('_')]
            name_ = label_class[0].upper()+label_class[1:]
            
            images_list.append(image_name)
            label_list.append(label_id_map[name_])
    
    all_data = pd.DataFrame.from_dict({'images':images_list,'labels':label_list})
    
    datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_dataframe(
        dataframe = all_data,
        x_col = 'images',
        y_col = 'labels',
        target_size = params['image_shape'],
        batch_size = 239,
        shuffle=False,
        class_mode = params['class_mode'])
    
    """
    
    datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory = os.path.join(params['data_dir'],'VUDU'),
        target_size = params['image_shape'],
        batch_size = 261,
        shuffle=False,
        class_mode = 'sparse')
    
    net = load_model(params)
    
    optimizer = tf.keras.optimizers.Adam(lr=params['learning_rate'])
    net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
    net.load_weights(os.path.join(params['model_dir'], 'tf_ckpt'))
    
    Y_pred = net.predict_generator(generator, steps=1)
    y_pred = np.argmax(Y_pred, axis=1)
           
    print(confusion_matrix(generator.classes, y_pred))
    target_names = ["acelera", "arranca", "avanza", "derecha", "pita", "izquierda","frena", "detente", "retrocede", "gira", ]
    print(classification_report(generator.classes, y_pred, target_names=target_names))

In [28]:
generator = vudu_preds(params)

Found 261 images belonging to 10 classes.
[[ 8  1  0  0  0  0  0 10  0 10]
 [ 0  3 26  0  0  1  0  0  0  0]
 [ 2  0 24  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  1  0 13]
 [ 0  0  0  0  9  0  0  0  0 22]
 [ 0  0  2  0  0 22  0  0  0  4]
 [ 1  0  2  0  0  1 22  2  0  0]
 [ 4  0  5  0  0  4  0 16  0  0]
 [ 0  1  0  0  0  0  0  5  3  1]
 [ 0  0  0  0  0  0  0  0  0 28]]
              precision    recall  f1-score   support

     acelera       0.53      0.28      0.36        29
     arranca       0.60      0.10      0.17        30
      avanza       0.41      0.92      0.56        26
     derecha       1.00      0.36      0.53        22
        pita       1.00      0.29      0.45        31
   izquierda       0.79      0.79      0.79        28
       frena       1.00      0.79      0.88        28
     detente       0.47      0.55      0.51        29
   retrocede       1.00      0.30      0.46        10
        gira       0.36      1.00      0.53        28

   micro avg       0.55      0.

In [17]:
preds = make_predictions(generator,params)
preds

array([9, 9, 7, 7, 7, 7, 0, 0, 9, 9, 0, 9, 7, 9, 0, 0, 7, 1, 0, 7, 9, 0,
       7, 7, 9, 0, 9, 9, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 5, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 9, 9, 9,
       9, 9, 9, 3, 9, 9, 9, 3, 9, 3, 7, 9, 3, 9, 3, 3, 3, 9, 3, 9, 9, 4,
       4, 9, 9, 9, 9, 9, 9, 9, 9, 4, 4, 9, 4, 9, 9, 9, 9, 4, 9, 9, 4, 4,
       9, 9, 9, 9, 4, 9, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 9,
       9, 5, 5, 5, 2, 5, 5, 5, 9, 5, 5, 5, 0, 5, 7, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 7, 6, 6, 2, 7, 7, 2,
       7, 5, 2, 7, 5, 7, 7, 7, 7, 2, 7, 7, 0, 0, 7, 5, 7, 0, 7, 7, 7, 0,
       5, 7, 2, 8, 7, 8, 7, 7, 9, 7, 8, 1, 7], dtype=int64)